In [ ]:
# get lambda density for free atoms
# calculate integral for specific distance between R_I, R_J
# fit integral or calculate directly?
# calculate alchemical binding potentials

In [4]:
import sys
sys.path.insert(0, '/home/misa/git_repositories/APDFT/prototyping/atomic_energies/')
import alchemy_tools2 as at

import numpy as np

In [2]:
def get_paths():
    pass

def load_cube_data(paths_cubes):
    """
    returns the data necessary to calculate the atomic energies from the cube-files
    for different lambda values
    
    paths_cubes: paths to cubes files
    densities: densities given in different cube files
    lam_vals: lambda value for cube file, obtained by parsing filename
    nuclei: charges and coordinates of the nuclei
    gpts: the gridpoints where density values are given
    """
    
    densities = []
    nuclei = None # nuclear charges and their positions
    gpts = None # gridpoints where density values are given
    h_matrix = np.zeros((3,3)) # needed for the calculation of the distance of the nuclei to the gridpoints with MIC
    
    for idx, path in enumerate(paths_cubes):
        cube = CUBE(path)
        
        densities.append(cube.data_scaled) # density
        
        if idx==len(paths_cubes)-1:
            nuclei = cube.atoms
            gpts = cube.get_grid()
            h_matrix = [cube.X*cube.NX, cube.Y*cube.NY, cube.Z*cube.NZ]
    
    return(np.array(densities), nuclei, gpts, h_matrix)

In [5]:
# load density files
paths_atom = get_paths()
densities, nucleus, gpts, h_matrix = load_cube_data(paths_atom)
lam_vals = []

TypeError: 'NoneType' object is not iterable

In [ ]:
# add density for lambda = 0

In [ ]:
# calculate averaged density
average_density = at.integrate_lambda_density(densities, lam_vals)

In [ ]:
dist_nuclei = []